In [1]:
# import stuff
import os
import numpy as np
import torch
import torch.utils.data as data
from itertools import product as product
import time

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import pandas as pd

# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

In [2]:
# make dirs
os.makedirs("weights", exist_ok=True)
os.makedirs("log", exist_ok=True)
input_size = 128
batch_size = 32

# Blazeface channels
channels = 64
"""
The original blazeface use channels=24. 
However, it was not enough to detect persons in our dataset.
Since detecting faces are quite a simple task compared to person detection, fewer channels were enabled.
"""

# Use focal loss or not
focal = False

# Use centernet-like Blazeface
center = False

# set up person only VOC dataset

In [3]:
# load files
vocpath = os.path.join("..", "VOCdevkit", "VOC2007")
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath, cls="person")

# extend with VOC2012
vocpath = "../VOCdevkit/VOC2012"
train_img_list2, train_anno_list2, _, _ = make_datapath_list(vocpath, cls="person", VOC2012=True)

train_img_list.extend(train_img_list2)
train_anno_list.extend(train_anno_list2)

# make Dataset
voc_classes = ['person']
color_mean = (104, 117, 123)  # (BGR)の色の平均値

print("trainlist: ", len(train_img_list))
print("vallist: ", len(val_img_list))

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(voc_classes)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase = "train", transform=transform, transform_anno = transform_anno)
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn, num_workers=8)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn, num_workers=8)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

val not found
trainlist:  6469
vallist:  2097


In [4]:
# check operation
batch_iterator = iter(dataloaders_dict["train"])  # iter
images, targets = next(batch_iterator)  # get first element
print(images.size())  # torch.Size([4, 3, 300, 300])
print(len(targets))
print(targets[1].shape)  # check targets

torch.Size([32, 3, 128, 128])
32
torch.Size([3, 5])


In [5]:
targets[1]

tensor([[0.4453, 0.4589, 0.8464, 1.0000, 0.0000],
        [0.4453, 0.5749, 0.6536, 1.0000, 0.0000],
        [0.8151, 0.7198, 0.8932, 0.9614, 0.0000]])

# test with ssd model.

In [6]:
if input_size==256:
    from utils.blazeface import SSD256 as SSD
    ssd_cfg = {
        'num_classes': 2,  # 背景クラスを含めた合計クラス数
        'input_size': input_size,  # 画像の入力サイズ
        'bbox_aspect_num': [4, 6],  # 出力するDBoxのアスペクト比の種類
        'feature_maps': [16, 8],  # 各sourceの画像サイズ
        'steps': [8, 16],  # DBOXの大きさを決める
        'min_sizes': [16, 32],  # DBOXの大きさを決める
        'max_sizes': [32, 100],  # DBOXの大きさを決める
        'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    }
elif input_size==128:
    from utils.blazeface import SSD
    ssd_cfg = {
        'num_classes': 2,  # 背景クラスを含めた合計クラス数
        'input_size': 128,  # 画像の入力サイズ
        'bbox_aspect_num': [4, 6],  # 出力するDBoxのアスペクト比の種類
        'feature_maps': [16, 8],  # 各sourceの画像サイズ
        'steps': [4, 8],  # DBOXの大きさを決める
        'min_sizes': [30, 60],  # DBOXの大きさを決める
        'max_sizes': [60, 128],  # DBOXの大きさを決める
        'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
        }

In [7]:
net = SSD(phase="train", cfg=ssd_cfg, channels=channels)

# SSDのweightsを設定

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# set inits for loc and conf
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

print("set weights!")

using: cuda:0
set weights!


In [8]:
print(net)

SSD(
  (blaze): BlazeFace(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=64)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (act): ReLU(inplace=True)
        (conv2): Sequential(
          (0): ReLU(inplace=True)
          (1): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=64)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): Conv2d(64, 64, kernel_size=(1, 1), strid

In [9]:
from utils.ssd_model import MultiBoxLoss

# define loss
criterion = MultiBoxLoss(jaccard_thresh=0.5,neg_pos=3, device=device, focal=focal)

# optim
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=1e-4, weight_decay=5e-4)

In [10]:
def get_current_lr(epoch):
    lr = 1e-4
    for i,lr_decay_epoch in enumerate([80,120]):
        if epoch >= lr_decay_epoch:
            lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    print("lr is:", lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [11]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("used device：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):
        
        adjust_learning_rate(optimizer, epoch)
        
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('train')
            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('val')
                else:
                    # 検証は5回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('Iteration {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log/log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/blazeface{}_'.format(input_size) +
                       str(epoch+1) + '.pth')

# start training here

In [ ]:
num_epochs = 150
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

used device： cuda:0
lr is: 0.0001
-------------
Epoch 1/150
-------------
train
Iteration 10 || Loss: 22.5487 || 10iter: 7.5143 sec.
Iteration 20 || Loss: 20.4253 || 10iter: 3.6372 sec.
Iteration 30 || Loss: 18.7865 || 10iter: 3.7234 sec.
Iteration 40 || Loss: 19.4787 || 10iter: 3.7046 sec.
Iteration 50 || Loss: 16.7488 || 10iter: 3.6960 sec.
Iteration 60 || Loss: 14.9106 || 10iter: 3.6627 sec.
Iteration 70 || Loss: 16.1222 || 10iter: 3.6477 sec.
Iteration 80 || Loss: 16.4054 || 10iter: 3.6974 sec.
Iteration 90 || Loss: 15.2272 || 10iter: 3.6138 sec.
Iteration 100 || Loss: 15.6662 || 10iter: 3.6454 sec.
Iteration 110 || Loss: 15.4372 || 10iter: 3.7520 sec.
Iteration 120 || Loss: 14.2358 || 10iter: 4.0816 sec.
Iteration 130 || Loss: 14.4491 || 10iter: 4.0365 sec.
Iteration 140 || Loss: 14.3126 || 10iter: 3.6645 sec.
Iteration 150 || Loss: 19.1626 || 10iter: 3.7097 sec.
Iteration 160 || Loss: 13.7796 || 10iter: 3.7511 sec.
Iteration 170 || Loss: 17.5071 || 10iter: 3.7056 sec.
Iteration 1

Iteration 1340 || Loss: 8.1670 || 10iter: 3.6865 sec.
Iteration 1350 || Loss: 10.7478 || 10iter: 3.7634 sec.
Iteration 1360 || Loss: 9.5503 || 10iter: 3.7305 sec.
Iteration 1370 || Loss: 8.5046 || 10iter: 3.6910 sec.
Iteration 1380 || Loss: 9.4814 || 10iter: 3.7139 sec.
Iteration 1390 || Loss: 8.8221 || 10iter: 3.6337 sec.
Iteration 1400 || Loss: 8.8724 || 10iter: 3.7073 sec.
Iteration 1410 || Loss: 8.7612 || 10iter: 3.5459 sec.
Iteration 1420 || Loss: 8.2808 || 10iter: 3.4160 sec.
-------------
epoch 7 || Epoch_TRAIN_Loss:1820.9165 ||Epoch_VAL_Loss:0.0000
timer:  77.9438 sec.
lr is: 0.0001
-------------
Epoch 8/150
-------------
train
Iteration 1430 || Loss: 9.0718 || 10iter: 6.7785 sec.
Iteration 1440 || Loss: 10.1622 || 10iter: 3.7324 sec.
Iteration 1450 || Loss: 9.8537 || 10iter: 3.7132 sec.
Iteration 1460 || Loss: 8.3607 || 10iter: 3.6899 sec.
Iteration 1470 || Loss: 8.3677 || 10iter: 3.7450 sec.
Iteration 1480 || Loss: 8.2059 || 10iter: 3.6585 sec.
Iteration 1490 || Loss: 11.7468

Iteration 2650 || Loss: 7.3811 || 10iter: 4.8936 sec.
Iteration 2660 || Loss: 6.9956 || 10iter: 3.6586 sec.
Iteration 2670 || Loss: 7.2370 || 10iter: 3.7618 sec.
Iteration 2680 || Loss: 9.3859 || 10iter: 3.6937 sec.
Iteration 2690 || Loss: 8.5226 || 10iter: 3.6454 sec.
Iteration 2700 || Loss: 6.6618 || 10iter: 3.6676 sec.
Iteration 2710 || Loss: 7.3833 || 10iter: 3.6418 sec.
Iteration 2720 || Loss: 7.6083 || 10iter: 3.7231 sec.
Iteration 2730 || Loss: 6.5915 || 10iter: 3.6188 sec.
Iteration 2740 || Loss: 7.0925 || 10iter: 3.6939 sec.
Iteration 2750 || Loss: 6.8639 || 10iter: 3.6755 sec.
Iteration 2760 || Loss: 6.4041 || 10iter: 3.7471 sec.
Iteration 2770 || Loss: 7.9480 || 10iter: 3.7545 sec.
Iteration 2780 || Loss: 6.3801 || 10iter: 3.6705 sec.
Iteration 2790 || Loss: 7.4054 || 10iter: 3.7307 sec.
Iteration 2800 || Loss: 7.2402 || 10iter: 3.7251 sec.
Iteration 2810 || Loss: 7.8042 || 10iter: 3.7182 sec.
Iteration 2820 || Loss: 8.3246 || 10iter: 3.6670 sec.
Iteration 2830 || Loss: 7.72

Iteration 3990 || Loss: 5.9829 || 10iter: 3.7686 sec.
Iteration 4000 || Loss: 5.7105 || 10iter: 3.6965 sec.
Iteration 4010 || Loss: 5.4220 || 10iter: 3.7337 sec.
Iteration 4020 || Loss: 7.3890 || 10iter: 3.6648 sec.
Iteration 4030 || Loss: 6.2232 || 10iter: 3.7293 sec.
Iteration 4040 || Loss: 6.4296 || 10iter: 3.7058 sec.
Iteration 4050 || Loss: 6.5877 || 10iter: 3.5883 sec.
Iteration 4060 || Loss: 6.4441 || 10iter: 3.1801 sec.
-------------
val
-------------
epoch 20 || Epoch_TRAIN_Loss:1328.0876 ||Epoch_VAL_Loss:443.7957
timer:  88.7218 sec.
lr is: 0.0001
-------------
Epoch 21/150
-------------
train
Iteration 4070 || Loss: 7.0428 || 10iter: 7.4494 sec.
Iteration 4080 || Loss: 6.1590 || 10iter: 3.5673 sec.
Iteration 4090 || Loss: 7.3442 || 10iter: 3.7159 sec.
Iteration 4100 || Loss: 6.8254 || 10iter: 3.6860 sec.
Iteration 4110 || Loss: 6.1635 || 10iter: 3.6744 sec.
Iteration 4120 || Loss: 6.0537 || 10iter: 3.6815 sec.
Iteration 4130 || Loss: 6.0340 || 10iter: 3.7479 sec.
Iteration 4

That's all :)